<a href="https://colab.research.google.com/github/sakamototaisei/python_colab/blob/main/AutoGluon%E3%81%AB%E3%82%88%E3%82%8B%E5%88%86%E9%A1%9E%E3%83%A2%E3%83%87%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AutoGluonとは**

## AutoGluonインストール



*   AutoGluonは、画像、テキスト、表示形式のデータセットを含む実世界のアプリケーションのための深層学習と機械学習を自動化する新しいオープンソースのAutoML
*   高精度なMLモデルを構築するには、データの前処理、特徴量抽出、ハイパーパラメーターの最適化、モデル選択といった多くの試行錯誤を伴うが、AutoGluonはこれらの判断を開発者に変わって、利用可能な計算資源を活用し実行時間内に最もよいモデルを見つけ出そうとする



In [83]:
pip install autogluon #①AutoGluonのインストール

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [84]:
pip install mxnet --upgrade #②MXNetのインストール

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
pip show autogluon #③インストールされているか確認

Name: autogluon
Version: 0.7.0
Summary: AutoML for Image, Text, and Tabular Data
Home-page: https://github.com/autogluon/autogluon
Author: AutoGluon Community
Author-email: 
License: Apache-2.0
Location: /usr/local/lib/python3.9/dist-packages
Requires: autogluon.core, autogluon.features, autogluon.multimodal, autogluon.tabular, autogluon.timeseries
Required-by: 


# **二値分類【Titanic】**

## データセットの取得

In [86]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## データの加工

In [87]:
# ①aliveカラムの削除
titanic = titanic.drop('alive', axis=1)
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,True


In [88]:
# ②説明変数と目的変数の作成
X = titanic.drop("survived",axis=1)
y = titanic["survived"]

In [89]:
# ③TrainデータとTestデータの作成
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [90]:
# ④目的変数データの割合の確認
print('Trainデータ\n', y_train.value_counts()/len(y_train))
print()
print('Testデータ\n', y_test.value_counts()/len(y_test))

Trainデータ
 0    0.616573
1    0.383427
Name: survived, dtype: float64

Testデータ
 0    0.614525
1    0.385475
Name: survived, dtype: float64


In [91]:
# ⑤説明変数と目的変数の結合
import pandas as pd
titanic_train = pd.concat([X_train, y_train], axis=1)

In [92]:
titanic_train.head()

,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alone,survived
502,3,female,NaN,0,0,7.6292,Q,Third,woman,False,NaN,Queenstown,True,0
464,3,male,NaN,0,0,8.0500,S,Third,man,True,NaN,Southampton,True,0
198,3,female,NaN,0,0,7.7500,Q,Third,woman,False,NaN,Queenstown,True,1
765,1,female,51.0,1,0,77.9583,S,First,woman,False,D,Southampton,False,1
421,3,male,21.0,0,0,7.7333,Q,Third,man,True,NaN,Queenstown,True,0


## AutoGluonによるモデルの作成

In [102]:
# ①AutoGluonの実行
from autogluon.tabular import TabularPredictor

predictor = TabularPredictor(label='survived', path='section2')
predictor.fit(train_data=titanic_train)

Beginning AutoGluon training ...
AutoGluon will save models to "section2/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    712
Train Data Columns: 13
Label Column: survived
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11509.44 MB
	Train Data (Original)  Memory Usage: 0.21 MB (0.0% of available memory)
	Inferring data type of each feature based on column valu

## モデルによる予測

In [94]:
# ①Testデータの目的変数を予測
y_pred = predictor.predict(X_test)
y_pred

153    0
752    0
610    0
200    0
310    1
      ..
96     0
440    1
75     0
575    0
143    0
Name: survived, Length: 179, dtype: int64

## モデルの評価

In [95]:
# ①モデルの予測精度を算出
# 引数：y_ture=実測値, y_pred=予測値, auxiliary_metrics=accuracy以外の評価指標を取得したい為True
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.7988826815642458
Evaluations on test data:
{
    "accuracy": 0.7988826815642458,
    "balanced_accuracy": 0.7634387351778656,
    "mcc": 0.5681204773909928,
    "f1": 0.7,
    "precision": 0.8235294117647058,
    "recall": 0.6086956521739131
}


# **Parameters/Attributes/Methodsの学習**

## Parameters of TabularPredicorの紹介

In [96]:
# ①TabularPredictorのParameters
predictor = TabularPredictor(label="survived",  # 予測に用いる目的変数のカラム名を入力します
                             problem_type='binary', # 回帰問題か分類問題の指定
                             eval_metric='accuracy', # 予測値を評価するための指標を選択する
                             path="section2", # モデル及び中間出力を保存するディレクトリーへのパス
                             verbosity=2, # モデルの学習が完了した後のアウトプットの情報量を制御するパラメータ
                             sample_weight='fare', # サンプルウェイトとして扱われるべきカラムを選択
                             weight_evaluation=False, # 検証データとテストデータの評価指標を計算する際にサンプルウェイトを考慮するかどうか
                             groups=None, # カラムを指定することで、そのカラムをバギングの為のデータ分割しようとする
                             )

## Attributesの紹介

①predictor.path  
②predictor.problem_type  
③predictor.eval_metric  
④predictor.label  
⑤predictor.feature_metadata.type_map_raw  
⑥predictor.feature_metadata.type_group_map_special  
⑦predictor.positive_class  
⑧predictor.class_labels  
⑨predictor.class_labels_internal  
⑩predictor.class_labels_internal_map

In [97]:
# ①全てのモデルが保存させれるディレクトリパスを出力する
predictor.path

'section2/'

In [98]:
# ②予測した問題のタイプについて出力する
predictor.problem_type

'binary'

In [99]:
# ③モデルの予測性能を表す評価指標を出力する
predictor.eval_metric

accuracy

In [100]:
# ④設定した目的変数を出力する
predictor.label

'survived'

In [103]:
# ⑤データは前処理後のカラムの推定データ型を出力する
predictor.feature_metadata.type_map_raw

{'pclass': 'int',
 'sex': 'int',
 'age': 'float',
 'sibsp': 'int',
 'parch': 'int',
 'fare': 'float',
 'adult_male': 'int',
 'alone': 'int',
 'embarked': 'category',
 'class': 'category',
 'who': 'category',
 'deck': 'category',
 'embark_town': 'category'}

In [104]:
# ⑥特別に生成されたカラムのデータ型を出力する
predictor.feature_metadata.type_group_map_special

defaultdict(list, {'bool': ['sex', 'adult_male', 'alone']})

In [105]:
# ⑦二値分類におけるポジティブクラスの出力を行う
predictor.positive_class

1

In [106]:
# ⑧クラスラベルを出力する
predictor.class_labels

[0, 1]

In [107]:
# ⑨クラスラベルと出力する
predictor.class_labels_internal

[0, 1]

In [108]:
# ⑩辞書形式による元のラベルと内部ラベルの対応関係が出力される
predictor.class_labels_internal_map

{0: 0, 1: 1}

## Mathodsの紹介

①predictor.distill(titanic_train)  
②predictor.evaluate(titanic_train)  
③predictor.evaluate_predictions(y_test,y_pred)  
④predictor.feature_importance(titanic_train)  
⑤predictor.features()  
⑥predictor.fit_summary()  
⑦predictor.get_model_best()  
⑧predictor.get_model_names()  
⑨predictor.leaderboard()  
⑩predictor1=TabularPredictor.load("section2")  
⑪y_pred_proba = predictor.predict_proba(X_test)z

In [109]:
# ①分解し、新しいモデルを作成する
predictor.distill(titanic_train)

Distilling with teacher='WeightedEnsemble_L2', teacher_preds=soft, augment_method=spunge ...
SPUNGE: Augmenting training data with 2845 synthetic samples for distillation...
Distilling with each of these student models: ['LightGBM_DSTL', 'NeuralNetMXNet_DSTL', 'CatBoost_DSTL', 'RandomForestMSE_DSTL', 'NeuralNetTorch_DSTL']
Fitting 5 L1 models ...
Fitting model: LightGBM_DSTL ...
	Note: model has different eval_metric than default.
	-0.1312	 = Validation score   (-mean_squared_error)
	0.67s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetMXNet_DSTL ...
		'fill_value'=!missing! is invalid. Expected a numerical value when imputing numerical data
Detailed Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/autogluon/core/trainer/abstract_trainer.py", line 1502, in _train_and_save
    model = self._train_single(X, y, model, X_val, y_val, total_resources=total_resources, **model_fit_kwargs)
  File "/usr/local/lib/python3.9/d

['LightGBM_DSTL',
 'CatBoost_DSTL',
 'RandomForestMSE_DSTL',
 'NeuralNetTorch_DSTL',
 'WeightedEnsemble_L2_DSTL']

In [110]:
# ①分類問題のため、accuracyなどの評価指標が出力される
# 与えられたデータセットと使用し、モデルによって評価された予測性能を出力する
predictor.evaluate(titanic_train)

Evaluation: accuracy on test data: 0.8764044943820225
Evaluations on test data:
{
    "accuracy": 0.8764044943820225,
    "balanced_accuracy": 0.8499086335077224,
    "mcc": 0.7392064899028196,
    "roc_auc": 0.9414753811109164,
    "f1": 0.8204081632653062,
    "precision": 0.9262672811059908,
    "recall": 0.7362637362637363
}


{'accuracy': 0.8764044943820225,
 'balanced_accuracy': 0.8499086335077224,
 'mcc': 0.7392064899028196,
 'roc_auc': 0.9414753811109164,
 'f1': 0.8204081632653062,
 'precision': 0.9262672811059908,
 'recall': 0.7362637362637363}

In [111]:
# 実測値と予測値を使用し、モデルによって評価された予測性能を出力する
predictor.evaluate_predictions(y_test, y_pred)

Evaluation: accuracy on test data: 0.7988826815642458
Evaluations on test data:
{
    "accuracy": 0.7988826815642458,
    "balanced_accuracy": 0.7634387351778656,
    "mcc": 0.5681204773909928,
    "f1": 0.7,
    "precision": 0.8235294117647058,
    "recall": 0.6086956521739131
}


{'accuracy': 0.7988826815642458,
 'balanced_accuracy': 0.7634387351778656,
 'mcc': 0.5681204773909928,
 'f1': 0.7,
 'precision': 0.8235294117647058,
 'recall': 0.6086956521739131}

In [112]:
# データセットに対してモデルによる特徴量重要度を抽出する
predictor.feature_importance(titanic_train)

Computing feature importance via permutation shuffling for 13 features using 712 rows with 5 shuffle sets...
	13.19s	= Expected runtime (2.64s per shuffle set)
	2.35s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
adult_male,0.104775,0.008165,4.390870e-06,5,0.121588,0.087963
pclass,0.068539,0.003635,9.462898e-07,5,0.076025,0.061054
age,0.038483,0.004283,1.811364e-05,5,0.047302,0.029664
fare,0.037640,0.003045,5.093989e-06,5,0.043910,0.031371
who,0.032303,0.007628,3.469710e-04,5,0.048010,0.016596
embark_town,0.020506,0.002551,2.817642e-05,5,0.025759,0.015252
deck,0.019944,0.004700,3.443229e-04,5,0.029622,0.010266
sex,0.017416,0.005403,9.822901e-04,5,0.028541,0.006290
sibsp,0.014045,0.004763,1.370110e-03,5,0.023852,0.004238
parch,0.013202,0.002551,1.593585e-04,5,0.018456,0.007949


In [113]:
# 元の学習データで指定された特徴量のリストを返す
predictor.features(feature_stage='original')

['pclass',
 'sex',
 'age',
 'sibsp',
 'parch',
 'fare',
 'embarked',
 'class',
 'who',
 'adult_male',
 'deck',
 'embark_town',
 'alone']

In [114]:
# データ前処理後の特徴量のリストを返す
predictor.features(feature_stage='transformed')

['pclass',
 'sex',
 'age',
 'sibsp',
 'parch',
 'fare',
 'adult_male',
 'alone',
 'embarked',
 'class',
 'who',
 'deck',
 'embark_town']

In [115]:
# 学習中に生成されたモデルに関する情報の要約を出力する
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                       model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0        WeightedEnsemble_L2   0.874126       0.125690   5.799646                0.000931           0.907730            2       True         14
1                   CatBoost   0.860140       0.008780   2.008960                0.008780           2.008960            1       True          7
2                   LightGBM   0.860140       0.009346   0.482572                0.009346           0.482572            1       True          4
3             NeuralNetTorch   0.853147       0.017828   1.436066                0.017828           1.436066            1       True         12
4       RandomForestMSE_DSTL   0.853147       0.091000   5.220878                0.091000           5.220878            1       True         17
5   WeightedEnsemble_L2_DSTL   0.853147       0.097755   6.053012         

{'model_types': {'KNeighborsUnif': 'KNNModel',
  'KNeighborsDist': 'KNNModel',
  'LightGBMXT': 'LGBModel',
  'LightGBM': 'LGBModel',
  'RandomForestGini': 'RFModel',
  'RandomForestEntr': 'RFModel',
  'CatBoost': 'CatBoostModel',
  'ExtraTreesGini': 'XTModel',
  'ExtraTreesEntr': 'XTModel',
  'NeuralNetFastAI': 'NNFastAiTabularModel',
  'XGBoost': 'XGBoostModel',
  'NeuralNetTorch': 'TabularNeuralNetTorchModel',
  'LightGBMLarge': 'LGBModel',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel',
  'LightGBM_DSTL': 'LGBModel',
  'CatBoost_DSTL': 'CatBoostModel',
  'RandomForestMSE_DSTL': 'RFModel',
  'NeuralNetTorch_DSTL': 'TabularNeuralNetTorchModel',
  'WeightedEnsemble_L2_DSTL': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif': 0.6293706293706294,
  'KNeighborsDist': 0.6433566433566433,
  'LightGBMXT': 0.8461538461538461,
  'LightGBM': 0.8601398601398601,
  'RandomForestGini': 0.7972027972027972,
  'RandomForestEntr': 0.8041958041958042,
  'CatBoost': 0.8601398601398601,

In [116]:
# バリデーションスコアにおける最良のモデル名を返す
predictor.get_model_best()

'WeightedEnsemble_L2'

In [117]:
# 学習されたモデルのリストを返す
predictor.get_model_names()

['KNeighborsUnif',
 'KNeighborsDist',
 'LightGBMXT',
 'LightGBM',
 'RandomForestGini',
 'RandomForestEntr',
 'CatBoost',
 'ExtraTreesGini',
 'ExtraTreesEntr',
 'NeuralNetFastAI',
 'XGBoost',
 'NeuralNetTorch',
 'LightGBMLarge',
 'WeightedEnsemble_L2',
 'LightGBM_DSTL',
 'CatBoost_DSTL',
 'RandomForestMSE_DSTL',
 'NeuralNetTorch_DSTL',
 'WeightedEnsemble_L2_DSTL']

In [118]:
# 学習で生成されたモデルに関する情報の要約をデータフレーム形式で出力する
# stack_levelが高いと検証スコアが高い傾向にある
predictor.leaderboard()

                       model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0        WeightedEnsemble_L2   0.874126       0.125690   5.799646                0.000931           0.907730            2       True         14
1                   CatBoost   0.860140       0.008780   2.008960                0.008780           2.008960            1       True          7
2                   LightGBM   0.860140       0.009346   0.482572                0.009346           0.482572            1       True          4
3             NeuralNetTorch   0.853147       0.017828   1.436066                0.017828           1.436066            1       True         12
4       RandomForestMSE_DSTL   0.853147       0.091000   5.220878                0.091000           5.220878            1       True         17
5   WeightedEnsemble_L2_DSTL   0.853147       0.097755   6.053012                0.000503           0.167053            2       True    

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.874126,0.125690,5.799646,0.000931,0.907730,2,True,14
1,CatBoost,0.860140,0.008780,2.008960,0.008780,2.008960,1,True,7
2,LightGBM,0.860140,0.009346,0.482572,0.009346,0.482572,1,True,4
3,NeuralNetTorch,0.853147,0.017828,1.436066,0.017828,1.436066,1,True,12
4,RandomForestMSE_DSTL,0.853147,0.091000,5.220878,0.091000,5.220878,1,True,17
5,WeightedEnsemble_L2_DSTL,0.853147,0.097755,6.053012,0.000503,0.167053,2,True,19
6,LightGBM_DSTL,0.846154,0.006252,0.665081,0.006252,0.665081,1,True,15
7,LightGBMLarge,0.846154,0.007515,0.517977,0.007515,0.517977,1,True,13
8,LightGBMXT,0.846154,0.008579,0.450396,0.008579,0.450396,1,True,3
9,NeuralNetTorch_DSTL,0.846154,0.016172,6.402230,0.016172,6.402230,1,True,18


In [119]:
# 前回の学習によって生成されたTabularPredictorオブジェクトをファイルからロードし、そのオプジェクトを返す
predictor1=TabularPredictor.load("section2")
predictor1

In [120]:
# 学習したモデルを使用し、予測されるクラスの確率を出力。クラスラベルは出力しない
y_pred_proba = predictor.predict_proba(X_test)
y_pred_proba

,0,1
153,0.706634,0.293366
752,0.679877,0.320123
610,0.625043,0.374957
200,0.714397,0.285603
310,0.298704,0.701296
...,...,...
96,0.662711,0.337289
440,0.410172,0.589828
75,0.710755,0.289245
575,0.725852,0.274148


# **多クラス分類【iris】**

## データセットの取得から実行

In [121]:
#①irisデータセットの取得からAutoGluonの実行
import seaborn as sns
iris = sns.load_dataset('iris')

X = iris.drop("species", axis=1)
y = iris["species"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

import pandas as pd
iris_train = pd.concat([X_train, y_train], axis=1)

from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label="species", path="section4")
predictor.fit(train_data=iris_train)

y_pred = predictor.predict(X_test)

perf = predictor.evaluate_predictions(y_true=y_test, 
                                      y_pred=y_pred, 
                                      auxiliary_metrics=True)

print(predictor.fit_summary())
print("ベストモデル：",predictor.get_model_best())

Beginning AutoGluon training ...
AutoGluon will save models to "section4/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    120
Train Data Columns: 4
Label Column: species
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	3 unique label values:  ['setosa', 'versicolor', 'virginica']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11566.61 MB
	Train Data (Original)  Memory Usage: 0.0 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. S

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0              CatBoost        1.0       0.001694  0.228167                0.001694           0.228167            1       True          8
1              LightGBM        1.0       0.001859  0.289691                0.001859           0.289691            1       True          5
2         LightGBMLarge        1.0       0.001940  0.400327                0.001940           0.400327            1       True         13
3            LightGBMXT        1.0       0.001993  0.278397                0.001993           0.278397            1       True          4
4   WeightedEnsemble_L2        1.0       0.002672  1.105094                0.000733           0.704767            2       True         14
5        KNeighborsDist        1.0       0.003940  0.010337                0.003940           0.010337        

## 予測精度の改善

In [122]:
#①irisデータセットの取得からAutoGluonの実行
import seaborn as sns
iris = sns.load_dataset('iris')

X = iris.drop("species", axis=1)
y = iris["species"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

import pandas as pd
iris_train = pd.concat([X_train, y_train], axis=1)

from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label="species", path="section4")
predictor.fit(train_data=iris_train,presets="best_quality")

y_pred = predictor.predict(X_test)

perf = predictor.evaluate_predictions(y_true=y_test, 
                                      y_pred=y_pred, 
                                      auxiliary_metrics=True)

print(predictor.fit_summary())
print("ベストモデル：",predictor.get_model_best())

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=5, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "section4/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    120
Train Data Columns: 4
Label Column: species
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	3 unique label values:  ['setosa', 'versicolor', 'virginica']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11535.24 MB
	Train 

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           LightGBM_BAG_L1   0.966667       0.009218  11.475046                0.009218          11.475046            1       True          5
1           CatBoost_BAG_L1   0.966667       0.010966  11.971380                0.010966          11.971380            1       True          8
2    NeuralNetFastAI_BAG_L1   0.966667       0.105108  25.941984                0.105108          25.941984            1       True          3
3       WeightedEnsemble_L2   0.966667       0.106229  27.026868                0.001121           1.084884            2       True         14
4         LightGBMXT_BAG_L1   0.958333       0.007088  12.041049                0.007088          12.041049            1       True          4
5      LightGBMLarge_BAG_L1   0.958333       0.010791  16.334108                

# **回帰モデルの作成**

## 【Tips】のデータセットの取得

In [123]:
# ①データセットの取得
import seaborn as sns
tips = sns.load_dataset('tips')
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


## 【Tips】の回帰モデルの作成

In [124]:
#①回帰モデルの作成
X = tips.drop("tip",axis=1)
y = tips["tip"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

import pandas as pd
tips_train = pd.concat([X_train,y_train],axis=1)

from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label="tip", path="section5-1")
predictor.fit(train_data=tips_train,presets="best_quality")

y_pred = predictor.predict(X_test)

perf = predictor.evaluate_predictions(y_true=y_test, 
                                      y_pred=y_pred, 
                                      auxiliary_metrics=True)

print(predictor.fit_summary())
print("ベストモデル：",predictor.get_model_best())

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=5, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "section5-1/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    195
Train Data Columns: 6
Label Column: tip
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (10.0, 1.0, 2.98492, 1.37488)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11530

*** Summary of fit() ***
Estimated performance of each model:
                     model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L2  -1.030583       0.566244  62.574533                0.000979           0.384074            2       True         12
1    NeuralNetTorch_BAG_L1  -1.049430       0.364637  19.999854                0.364637          19.999854            1       True         10
2   NeuralNetFastAI_BAG_L1  -1.054587       0.170049  29.332963                0.170049          29.332963            1       True          8
3          LightGBM_BAG_L1  -1.083299       0.030579  12.857641                0.030579          12.857641            1       True          4
4        LightGBMXT_BAG_L1  -1.087210       0.028568  11.689847                0.028568          11.689847            1       True          3
5          CatBoost_BAG_L1  -1.111892       0.025351  14.215774                0.02535

## 【Diamonds】データセットの取得

In [125]:
# ①データセットの取得
import seaborn as sns
diamonds = sns.load_dataset('diamonds')
diamonds

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


## 【Diamonds】回帰モデルの作成

In [126]:
#①回帰モデルの作成
X = diamonds.drop("price",axis=1)
y = diamonds["price"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

import pandas as pd
diamonds_train = pd.concat([X_train,y_train],axis=1)

from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label="price", path="section5-2")
# time_limitを設定することで、学習する時間を決めることができる
predictor.fit(train_data=diamonds_train,presets="best_quality",time_limit=15)

y_pred = predictor.predict(X_test)

perf = predictor.evaluate_predictions(y_true=y_test, 
                                      y_pred=y_pred, 
                                      auxiliary_metrics=True)

print(predictor.fit_summary())
print("ベストモデル：",predictor.get_model_best())

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 15s
AutoGluon will save models to "section5-2/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    43152
Train Data Columns: 9
Label Column: price
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (18823, 327, 3933.6967, 3991.44248)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory

*** Summary of fit() ***
Estimated performance of each model:
                   model    score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      LightGBMXT_BAG_L1  -620.404566       1.023201  29.879183                1.023201          29.879183            1       True          3
1    WeightedEnsemble_L2  -620.404566       1.024415  30.330589                0.001214           0.451406            2       True          4
2  KNeighborsUnif_BAG_L1 -1461.091419       0.759190   0.166143                0.759190           0.166143            1       True          1
3  KNeighborsDist_BAG_L1 -1470.977689       0.745932   0.281890                0.745932           0.281890            1       True          2
Number of models trained: 4
Types of models trained:
{'StackerEnsembleModel_KNN', 'WeightedEnsembleModel', 'StackerEnsembleModel_LGB'}
Bagging used: True  (with 8 folds)
Multi-layer stack-ensembling used: False 
Feature Metadata